# 스케일링_하이퍼파라미터튜닝

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import koreanize_matplotlib

In [2]:
data = pd.read_csv("./data/salary2.csv")

In [3]:
from sklearn.model_selection import train_test_split

In [4]:
train, test = train_test_split(data, test_size=0.4, random_state=42, stratify=data['class'])

In [5]:
train = train.drop_duplicates()
test = test.drop_duplicates()
train = train.dropna()
test = test.dropna()
train = train.reset_index(drop=True)
test = test.reset_index(drop=True)

In [6]:
from sklearn.preprocessing import LabelEncoder

In [7]:
le = LabelEncoder()
le.fit(train['class'])
train['class'] = le.transform(train['class'])
test['class'] = le.transform(test['class'])

In [8]:
train.head()

,age,workclass,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,class
0,18,Private,Some-college,10,Never-married,Transport-moving,Own-child,White,Male,2907,0,30,United-States,0
1,31,Private,11th,7,Never-married,Craft-repair,Not-in-family,White,Male,0,2001,40,United-States,0
2,34,Private,Bachelors,13,Never-married,Other-service,Not-in-family,White,Female,0,0,35,United-States,0
3,59,Private,11th,7,Married-civ-spouse,Adm-clerical,Husband,White,Male,0,0,40,United-States,0
4,41,Private,Prof-school,15,Married-civ-spouse,Prof-specialty,Wife,White,Female,0,0,40,United-States,1


In [9]:
test.head()

,age,workclass,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,class
0,52,Private,Bachelors,13,Divorced,Craft-repair,Not-in-family,White,Male,0,0,40,United-States,0
1,41,Private,HS-grad,9,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,40,United-States,0
2,65,Without-pay,7th-8th,4,Widowed,Farming-fishing,Unmarried,White,Female,0,0,50,United-States,0
3,57,Private,Some-college,10,Widowed,Tech-support,Not-in-family,White,Female,0,0,16,United-States,0
4,24,Private,Bachelors,13,Never-married,Tech-support,Not-in-family,White,Female,3674,0,40,United-States,0


# 숫자 컬럼 스케일링
# 📊 스케일러(Scaler) 종류별 차이 정리

## 1️⃣ StandardScaler (표준화)

| 항목 | 내용 |
|------|------|
| 변환 방식 | (값 - 평균) / 표준편차 |
| 결과 범위 | 평균 0, 표준편차 1 |
| 이상치 영향 | 큼 |
| 사용 추천 모델 | 로지스틱회귀, SVM, KNN, 신경망 |
| 특징 | 가장 많이 사용되는 기본 스케일링 |

📌 공식  
z = (x - μ) / σ

---

## 2️⃣ MinMaxScaler (정규화)

| 항목 | 내용 |
|------|------|
| 변환 방식 | (값 - 최소값) / (최대값 - 최소값) |
| 결과 범위 | 0 ~ 1 (기본 설정) |
| 이상치 영향 | 매우 큼 |
| 사용 추천 모델 | 신경망, 거리 기반 모델 |
| 특징 | 값의 범위를 일정하게 맞춰줌 |

📌 공식  
x' = (x - min) / (max - min)

---

## 3️⃣ RobustScaler (이상치 강건 스케일링)

| 항목 | 내용 |
|------|------|
| 변환 방식 | (값 - 중앙값) / IQR |
| 결과 범위 | 고정 범위 없음 |
| 이상치 영향 | 적음 |
| 사용 추천 모델 | 이상치가 많은 데이터 |
| 특징 | 중앙값과 사분위수를 사용 |

📌 공식  
x' = (x - median) / (Q3 - Q1)

---

## 4️⃣ MaxAbsScaler

| 항목 | 내용 |
|------|------|
| 변환 방식 | 값을 절대 최대값으로 나눔 |
| 결과 범위 | -1 ~ 1 |
| 이상치 영향 | 있음 |
| 사용 추천 모델 | 희소행렬(Sparse Matrix) |
| 특징 | 0을 유지함 (희소데이터에 적합) |

📌 공식  
x' = x / |max|

---

# 🔥 한눈에 비교

| 스케일러 | 평균 0 | 범위 고정 | 이상치 영향 | 가장 많이 쓰는 곳 |
|-----------|--------|-----------|-------------|----------------|
| Standard | O | X | 큼 | 선형모델, SVM |
| MinMax | X | O (0~1) | 매우 큼 | 신경망 |
| Robust | X | X | 적음 | 이상치 많은 데이터 |
| MaxAbs | X | O (-1~1) | 있음 | Sparse 데이터 |

---

# 🎯 실무 선택 가이드

| 상황 | 추천 |
|------|------|
| 일반적인 ML | StandardScaler |
| 값 범위를 0~1로 맞추고 싶을 때 | MinMaxScaler |
| 이상치가 많을 때 | RobustScaler |
| 희소행렬 사용할 때 | MaxAbsScaler |

---

# 📌 중요한 포인트

✔ Tree 모델 (RandomForest, XGBoost, LightGBM)은  
→ **스케일링 필요 없음**

✔ 거리 기반 모델(KNN, SVM, KMeans)은  
→ **스케일링 필수**

✔ 항상 train 데이터로 fit 후 test에는 transform만 사용

```python
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)
```

* standard scaler

In [11]:
X_train = train.drop('class', axis=1)
y_train = train['class']
X_test = test.drop('class', axis=1)
y_test = test['class']

In [14]:
num_cols = X_train.select_dtypes(exclude="object").columns
num_cols

Index(['age', 'education-num', 'capital-gain', 'capital-loss',
       'hours-per-week'],
      dtype='object')

In [10]:
from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler

In [17]:
scaler = StandardScaler()
X_train_std = scaler.fit_transform(X_train[num_cols])
X_test_std = scaler.transform(X_test[num_cols])
X_train_std = pd.DataFrame(X_train_std, index=X_train.index, columns=num_cols)
X_test_std = pd.DataFrame(X_test_std, index=X_test.index, columns=num_cols)
display(X_train_std)
display(X_test_std)

,age,education-num,capital-gain,capital-loss,hours-per-week
0,-1.592436,-0.050564,0.239364,-0.231949,-0.903514
1,-0.615677,-1.198794,-0.152487,4.447626,-0.095507
2,-0.390271,1.097666,-0.152487,-0.231949,-0.499511
3,1.488111,-1.198794,-0.152487,-0.231949,-0.095507
4,0.135676,1.863153,-0.152487,-0.231949,-0.095507
...,...,...,...,...,...
24505,2.014058,-2.347025,-0.152487,-0.231949,-0.095507
24506,0.661623,1.097666,-0.152487,-0.231949,-0.095507
24507,0.210811,-0.050564,-0.152487,-0.231949,-0.095507
24508,0.586487,1.480410,-0.152487,-0.231949,-0.499511


,age,education-num,capital-gain,capital-loss,hours-per-week
0,0.962164,1.097666,-0.152487,-0.231949,-0.095507
1,0.135676,-0.433307,-0.152487,-0.231949,-0.095507
2,1.938923,-2.347025,-0.152487,-0.231949,0.712500
3,1.337840,-0.050564,-0.152487,-0.231949,-2.034724
4,-1.141624,1.097666,0.342752,-0.231949,-0.095507
...,...,...,...,...,...
16659,-0.465407,-0.433307,-0.152487,-0.231949,-1.711521
16660,-0.765948,1.097666,0.474312,-0.231949,-0.095507
16661,0.887029,1.097666,-0.152487,3.678221,2.328514
16662,-0.390271,1.097666,0.831251,-0.231949,1.520507


In [18]:
cat_cols = X_train.select_dtypes(include="object").columns
cat_cols

Index(['workclass', 'education', 'marital-status', 'occupation',
       'relationship', 'race', 'sex', 'native-country'],
      dtype='object')

In [25]:
X_train_std = pd.concat([X_train[cat_cols], X_train_std], axis=1)
X_test_std = pd.concat([X_test[cat_cols], X_test_std], axis=1)
display(X_train_std)
display(X_test_std)

,workclass,education,marital-status,occupation,relationship,race,sex,native-country,age,education-num,capital-gain,capital-loss,hours-per-week
0,Private,Some-college,Never-married,Transport-moving,Own-child,White,Male,United-States,-1.592436,-0.050564,0.239364,-0.231949,-0.903514
1,Private,11th,Never-married,Craft-repair,Not-in-family,White,Male,United-States,-0.615677,-1.198794,-0.152487,4.447626,-0.095507
2,Private,Bachelors,Never-married,Other-service,Not-in-family,White,Female,United-States,-0.390271,1.097666,-0.152487,-0.231949,-0.499511
3,Private,11th,Married-civ-spouse,Adm-clerical,Husband,White,Male,United-States,1.488111,-1.198794,-0.152487,-0.231949,-0.095507
4,Private,Prof-school,Married-civ-spouse,Prof-specialty,Wife,White,Female,United-States,0.135676,1.863153,-0.152487,-0.231949,-0.095507
...,...,...,...,...,...,...,...,...,...,...,...,...,...
24505,Private,7th-8th,Divorced,Other-service,Other-relative,Asian-Pac-Islander,Female,Philippines,2.014058,-2.347025,-0.152487,-0.231949,-0.095507
24506,Self-emp-not-inc,Bachelors,Married-civ-spouse,Exec-managerial,Husband,Asian-Pac-Islander,Male,Japan,0.661623,1.097666,-0.152487,-0.231949,-0.095507
24507,Private,Some-college,Divorced,Prof-specialty,Unmarried,White,Female,United-States,0.210811,-0.050564,-0.152487,-0.231949,-0.095507
24508,Local-gov,Masters,Never-married,Prof-specialty,Not-in-family,White,Male,United-States,0.586487,1.480410,-0.152487,-0.231949,-0.499511


,workclass,education,marital-status,occupation,relationship,race,sex,native-country,age,education-num,capital-gain,capital-loss,hours-per-week
0,Private,Bachelors,Divorced,Craft-repair,Not-in-family,White,Male,United-States,0.962164,1.097666,-0.152487,-0.231949,-0.095507
1,Private,HS-grad,Married-civ-spouse,Exec-managerial,Husband,White,Male,United-States,0.135676,-0.433307,-0.152487,-0.231949,-0.095507
2,Without-pay,7th-8th,Widowed,Farming-fishing,Unmarried,White,Female,United-States,1.938923,-2.347025,-0.152487,-0.231949,0.712500
3,Private,Some-college,Widowed,Tech-support,Not-in-family,White,Female,United-States,1.337840,-0.050564,-0.152487,-0.231949,-2.034724
4,Private,Bachelors,Never-married,Tech-support,Not-in-family,White,Female,United-States,-1.141624,1.097666,0.342752,-0.231949,-0.095507
...,...,...,...,...,...,...,...,...,...,...,...,...,...
16659,Private,HS-grad,Divorced,Handlers-cleaners,Own-child,White,Male,United-States,-0.465407,-0.433307,-0.152487,-0.231949,-1.711521
16660,Private,Bachelors,Never-married,Prof-specialty,Not-in-family,White,Male,United-States,-0.765948,1.097666,0.474312,-0.231949,-0.095507
16661,Self-emp-not-inc,Bachelors,Married-civ-spouse,Exec-managerial,Husband,White,Male,United-States,0.887029,1.097666,-0.152487,3.678221,2.328514
16662,Private,Bachelors,Married-civ-spouse,Sales,Husband,White,Male,United-States,-0.390271,1.097666,0.831251,-0.231949,1.520507


* MinMaxscaler

In [26]:
mms = MinMaxScaler()
X_train_mms = mms.fit_transform(X_train[num_cols])
X_test_mms = mms.transform(X_test[num_cols])
X_train_mms = pd.DataFrame(X_train_mms, index=X_train.index, columns=num_cols)
X_test_mms = pd.DataFrame(X_test_mms, index=X_test.index, columns=num_cols)
X_train_mms = pd.concat([X_train[cat_cols], X_train_mms], axis=1)
X_test_mms = pd.concat([X_test[cat_cols], X_test_mms], axis=1)
display(X_train_mms)
display(X_test_mms)

,workclass,education,marital-status,occupation,relationship,race,sex,native-country,age,education-num,capital-gain,capital-loss,hours-per-week
0,Private,Some-college,Never-married,Transport-moving,Own-child,White,Male,United-States,0.013699,0.600000,0.02907,0.000000,0.295918
1,Private,11th,Never-married,Craft-repair,Not-in-family,White,Male,United-States,0.191781,0.400000,0.00000,0.459366,0.397959
2,Private,Bachelors,Never-married,Other-service,Not-in-family,White,Female,United-States,0.232877,0.800000,0.00000,0.000000,0.346939
3,Private,11th,Married-civ-spouse,Adm-clerical,Husband,White,Male,United-States,0.575342,0.400000,0.00000,0.000000,0.397959
4,Private,Prof-school,Married-civ-spouse,Prof-specialty,Wife,White,Female,United-States,0.328767,0.933333,0.00000,0.000000,0.397959
...,...,...,...,...,...,...,...,...,...,...,...,...,...
24505,Private,7th-8th,Divorced,Other-service,Other-relative,Asian-Pac-Islander,Female,Philippines,0.671233,0.200000,0.00000,0.000000,0.397959
24506,Self-emp-not-inc,Bachelors,Married-civ-spouse,Exec-managerial,Husband,Asian-Pac-Islander,Male,Japan,0.424658,0.800000,0.00000,0.000000,0.397959
24507,Private,Some-college,Divorced,Prof-specialty,Unmarried,White,Female,United-States,0.342466,0.600000,0.00000,0.000000,0.397959
24508,Local-gov,Masters,Never-married,Prof-specialty,Not-in-family,White,Male,United-States,0.410959,0.866667,0.00000,0.000000,0.346939


,workclass,education,marital-status,occupation,relationship,race,sex,native-country,age,education-num,capital-gain,capital-loss,hours-per-week
0,Private,Bachelors,Divorced,Craft-repair,Not-in-family,White,Male,United-States,0.479452,0.800000,0.000000,0.000000,0.397959
1,Private,HS-grad,Married-civ-spouse,Exec-managerial,Husband,White,Male,United-States,0.328767,0.533333,0.000000,0.000000,0.397959
2,Without-pay,7th-8th,Widowed,Farming-fishing,Unmarried,White,Female,United-States,0.657534,0.200000,0.000000,0.000000,0.500000
3,Private,Some-college,Widowed,Tech-support,Not-in-family,White,Female,United-States,0.547945,0.600000,0.000000,0.000000,0.153061
4,Private,Bachelors,Never-married,Tech-support,Not-in-family,White,Female,United-States,0.095890,0.800000,0.036740,0.000000,0.397959
...,...,...,...,...,...,...,...,...,...,...,...,...,...
16659,Private,HS-grad,Divorced,Handlers-cleaners,Own-child,White,Male,United-States,0.219178,0.533333,0.000000,0.000000,0.193878
16660,Private,Bachelors,Never-married,Prof-specialty,Not-in-family,White,Male,United-States,0.164384,0.800000,0.046500,0.000000,0.397959
16661,Self-emp-not-inc,Bachelors,Married-civ-spouse,Exec-managerial,Husband,White,Male,United-States,0.465753,0.800000,0.000000,0.383838,0.704082
16662,Private,Bachelors,Married-civ-spouse,Sales,Husband,White,Male,United-States,0.232877,0.800000,0.072981,0.000000,0.602041


* Robustscaler

In [27]:
rs = RobustScaler()
X_train_rs = rs.fit_transform(X_train[num_cols])
X_test_rs = rs.transform(X_test[num_cols])
X_train_rs = pd.DataFrame(X_train_rs, index=X_train.index, columns=num_cols)
X_test_rs = pd.DataFrame(X_test_rs, index=X_test.index, columns=num_cols)
X_train_rs = pd.concat([X_train[cat_cols], X_train_rs], axis=1)
X_test_rs = pd.concat([X_test[cat_cols], X_test_rs], axis=1)
display(X_train_rs)
display(X_test_rs)

,workclass,education,marital-status,occupation,relationship,race,sex,native-country,age,education-num,capital-gain,capital-loss,hours-per-week
0,Private,Some-college,Never-married,Transport-moving,Own-child,White,Male,United-States,-1.052632,0.00,2907.0,0.0,-2.0
1,Private,11th,Never-married,Craft-repair,Not-in-family,White,Male,United-States,-0.368421,-0.75,0.0,2001.0,0.0
2,Private,Bachelors,Never-married,Other-service,Not-in-family,White,Female,United-States,-0.210526,0.75,0.0,0.0,-1.0
3,Private,11th,Married-civ-spouse,Adm-clerical,Husband,White,Male,United-States,1.105263,-0.75,0.0,0.0,0.0
4,Private,Prof-school,Married-civ-spouse,Prof-specialty,Wife,White,Female,United-States,0.157895,1.25,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
24505,Private,7th-8th,Divorced,Other-service,Other-relative,Asian-Pac-Islander,Female,Philippines,1.473684,-1.50,0.0,0.0,0.0
24506,Self-emp-not-inc,Bachelors,Married-civ-spouse,Exec-managerial,Husband,Asian-Pac-Islander,Male,Japan,0.526316,0.75,0.0,0.0,0.0
24507,Private,Some-college,Divorced,Prof-specialty,Unmarried,White,Female,United-States,0.210526,0.00,0.0,0.0,0.0
24508,Local-gov,Masters,Never-married,Prof-specialty,Not-in-family,White,Male,United-States,0.473684,1.00,0.0,0.0,-1.0


,workclass,education,marital-status,occupation,relationship,race,sex,native-country,age,education-num,capital-gain,capital-loss,hours-per-week
0,Private,Bachelors,Divorced,Craft-repair,Not-in-family,White,Male,United-States,0.736842,0.75,0.0,0.0,0.0
1,Private,HS-grad,Married-civ-spouse,Exec-managerial,Husband,White,Male,United-States,0.157895,-0.25,0.0,0.0,0.0
2,Without-pay,7th-8th,Widowed,Farming-fishing,Unmarried,White,Female,United-States,1.421053,-1.50,0.0,0.0,2.0
3,Private,Some-college,Widowed,Tech-support,Not-in-family,White,Female,United-States,1.000000,0.00,0.0,0.0,-4.8
4,Private,Bachelors,Never-married,Tech-support,Not-in-family,White,Female,United-States,-0.736842,0.75,3674.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
16659,Private,HS-grad,Divorced,Handlers-cleaners,Own-child,White,Male,United-States,-0.263158,-0.25,0.0,0.0,-4.0
16660,Private,Bachelors,Never-married,Prof-specialty,Not-in-family,White,Male,United-States,-0.473684,0.75,4650.0,0.0,0.0
16661,Self-emp-not-inc,Bachelors,Married-civ-spouse,Exec-managerial,Husband,White,Male,United-States,0.684211,0.75,0.0,1672.0,6.0
16662,Private,Bachelors,Married-civ-spouse,Sales,Husband,White,Male,United-States,-0.210526,0.75,7298.0,0.0,4.0


* 문자컬럼 onehot

In [30]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder

In [31]:
def to_ohe(X_train, X_test):
    cat_cols = X_train.select_dtypes(include='object').columns
    ct = ColumnTransformer(transformers=[
        ('ohe', OneHotEncoder(drop='first', handle_unknown='ignore', sparse_output=False),
         cat_cols)], remainder='passthrough')
    ct.set_output(transform='pandas')
    X_train = ct.fit_transform(X_train)
    X_test = ct.transform(X_test)
    return X_train, X_test

In [32]:
X_train_std, X_test_std = to_ohe(X_train_std, X_test_std)
X_train_mms, X_test_mms = to_ohe(X_train_mms, X_test_mms)
X_train_rs, X_test_rs = to_ohe(X_train_rs, X_test_rs)
display(X_train_std)
display(X_train_mms)
display(X_train_rs)

,ohe__workclass_ Local-gov,ohe__workclass_ Private,ohe__workclass_ Self-emp-inc,ohe__workclass_ Self-emp-not-inc,ohe__workclass_ State-gov,ohe__workclass_ Without-pay,ohe__education_ 11th,ohe__education_ 12th,ohe__education_ 1st-4th,ohe__education_ 5th-6th,...,ohe__native-country_ Thailand,ohe__native-country_ Trinadad&Tobago,ohe__native-country_ United-States,ohe__native-country_ Vietnam,ohe__native-country_ Yugoslavia,remainder__age,remainder__education-num,remainder__capital-gain,remainder__capital-loss,remainder__hours-per-week
0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,-1.592436,-0.050564,0.239364,-0.231949,-0.903514
1,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,-0.615677,-1.198794,-0.152487,4.447626,-0.095507
2,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,-0.390271,1.097666,-0.152487,-0.231949,-0.499511
3,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,1.488111,-1.198794,-0.152487,-0.231949,-0.095507
4,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.135676,1.863153,-0.152487,-0.231949,-0.095507
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24505,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,2.014058,-2.347025,-0.152487,-0.231949,-0.095507
24506,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.661623,1.097666,-0.152487,-0.231949,-0.095507
24507,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.210811,-0.050564,-0.152487,-0.231949,-0.095507
24508,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.586487,1.480410,-0.152487,-0.231949,-0.499511


,ohe__workclass_ Local-gov,ohe__workclass_ Private,ohe__workclass_ Self-emp-inc,ohe__workclass_ Self-emp-not-inc,ohe__workclass_ State-gov,ohe__workclass_ Without-pay,ohe__education_ 11th,ohe__education_ 12th,ohe__education_ 1st-4th,ohe__education_ 5th-6th,...,ohe__native-country_ Thailand,ohe__native-country_ Trinadad&Tobago,ohe__native-country_ United-States,ohe__native-country_ Vietnam,ohe__native-country_ Yugoslavia,remainder__age,remainder__education-num,remainder__capital-gain,remainder__capital-loss,remainder__hours-per-week
0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.013699,0.600000,0.02907,0.000000,0.295918
1,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.191781,0.400000,0.00000,0.459366,0.397959
2,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.232877,0.800000,0.00000,0.000000,0.346939
3,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.575342,0.400000,0.00000,0.000000,0.397959
4,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.328767,0.933333,0.00000,0.000000,0.397959
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24505,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.671233,0.200000,0.00000,0.000000,0.397959
24506,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.424658,0.800000,0.00000,0.000000,0.397959
24507,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.342466,0.600000,0.00000,0.000000,0.397959
24508,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.410959,0.866667,0.00000,0.000000,0.346939


,ohe__workclass_ Local-gov,ohe__workclass_ Private,ohe__workclass_ Self-emp-inc,ohe__workclass_ Self-emp-not-inc,ohe__workclass_ State-gov,ohe__workclass_ Without-pay,ohe__education_ 11th,ohe__education_ 12th,ohe__education_ 1st-4th,ohe__education_ 5th-6th,...,ohe__native-country_ Thailand,ohe__native-country_ Trinadad&Tobago,ohe__native-country_ United-States,ohe__native-country_ Vietnam,ohe__native-country_ Yugoslavia,remainder__age,remainder__education-num,remainder__capital-gain,remainder__capital-loss,remainder__hours-per-week
0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,-1.052632,0.00,2907.0,0.0,-2.0
1,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,-0.368421,-0.75,0.0,2001.0,0.0
2,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,-0.210526,0.75,0.0,0.0,-1.0
3,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,1.105263,-0.75,0.0,0.0,0.0
4,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.157895,1.25,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24505,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.473684,-1.50,0.0,0.0,0.0
24506,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.526316,0.75,0.0,0.0,0.0
24507,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.210526,0.00,0.0,0.0,0.0
24508,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.473684,1.00,0.0,0.0,-1.0


# 다른 스케일의 데이터로 분석해보기

In [36]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import classification_report

In [35]:
X_trains = (X_train_std, X_train_mms, X_train_rs)
tests = (X_test_std, X_test_mms, X_test_rs)

In [46]:
for train, test in zip(trains, tests):
    dtc = DecisionTreeClassifier(criterion='log_loss', max_depth=10, class_weight='balanced', random_state=10)
    dtc.fit(train, y_train)
    pred = dtc.predict(test)
    print(classification_report(y_test, pred))

              precision    recall  f1-score   support

           0       0.94      0.79      0.86     12463
           1       0.58      0.84      0.69      4201

    accuracy                           0.81     16664
   macro avg       0.76      0.82      0.77     16664
weighted avg       0.85      0.81      0.82     16664

              precision    recall  f1-score   support

           0       0.94      0.79      0.86     12463
           1       0.58      0.84      0.69      4201

    accuracy                           0.81     16664
   macro avg       0.76      0.82      0.77     16664
weighted avg       0.85      0.81      0.82     16664

              precision    recall  f1-score   support

           0       0.94      0.79      0.86     12463
           1       0.58      0.84      0.69      4201

    accuracy                           0.81     16664
   macro avg       0.76      0.82      0.77     16664
weighted avg       0.85      0.81      0.82     16664



# 🎯 (salary2) 하이퍼파라미터 튜닝 + GridSearch/RandomSearch + StratifiedKFold

---

## 1) 하이퍼파라미터 튜닝이란?

| 구분 | 의미 | 예시 |
|---|---|---|
| 파라미터(Parameter) | 모델이 학습하면서 자동으로 찾는 값 | 회귀계수, 트리 분기 기준 등 |
| 하이퍼파라미터(Hyperparameter) | 사람이 미리 정해줘야 하는 설정값 | `max_depth`, `C`, `n_estimators` |

👉 **튜닝**은 하이퍼파라미터를 여러 값으로 바꿔보면서 **가장 성능이 좋은 조합을 찾는 과정**

---

## 2) GridSearch vs RandomSearch

| 방법 | 아이디어 | 장점 | 단점 | 추천 상황 |
|---|---|---|---|---|
| GridSearch | 가능한 조합을 **전부** 시도 | 범위가 작으면 가장 확실 | 조합이 커지면 매우 느림 | 파라미터 후보가 적을 때 |
| RandomSearch | 범위 안에서 **일부를 랜덤** 샘플링 | 빠르고 실무에서 효율적 | 최적 조합을 놓칠 수 있음 | 범위가 넓을 때(실무 기본) |

---

## 3) 교차검증(CV)이 필요한 이유

| 한 번 분할(train/test)만 쓰면? | 교차검증을 쓰면? |
|---|---|
| 운이 좋으면 성능이 과대평가될 수 있음 | 여러 번 나눠 평균 성능 → 더 안정적 |

---

## 4) StratifiedKFold는 언제 쓰나? ⭐

### ✅ 분류(Classification) + 클래스 불균형일 때 필수
- 예: (salary2가) `high_salary(0/1)` 같은 타깃이면  
  폴드마다 0/1 비율이 비슷하게 유지되어야 평가가 공정함.

| 분할 방식 | 특징 |
|---|---|
| KFold | 그냥 나눔(클래스 비율 깨질 수 있음) |
| StratifiedKFold | **각 폴드의 클래스 비율 유지** (불균형 데이터에 적합) |

> ⚠️ salary2가 **회귀(연봉 숫자 예측)**라면 StratifiedKFold가 아니라 **KFold**를 사용해야 함.

---

# randomSearch

In [50]:
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV, StratifiedKFold
from sklearn.ensemble import RandomForestClassifier

In [52]:
rfc = RandomForestClassifier(n_jobs=-1, random_state=10)

In [49]:
# kfold 교차검증 정의하기
cv_reg = StratifiedKFold()

In [51]:
param_grid = dict(max_depth=[1, 5, 10], min_samples_split=[2,10], n_estimators=[100, 300, 500])
param_grid

{'max_depth': [1, 5, 10],
 'min_samples_split': [2, 10],
 'n_estimators': [100, 300, 500]}

In [53]:
randcv = RandomizedSearchCV(estimator=rfc, param_distributions=param_grid, cv=cv_reg, n_jobs=-1)
randcv.fit(X_train_std, y_train)
pred = randcv.predict(X_test_std)
print(classification_report(y_test, pred))
print("best_params: ", randcv.best_params_)

              precision    recall  f1-score   support

           0       0.86      0.96      0.91     12463
           1       0.81      0.55      0.66      4201

    accuracy                           0.85     16664
   macro avg       0.84      0.75      0.78     16664
weighted avg       0.85      0.85      0.84     16664

best_params:  {'n_estimators': 100, 'min_samples_split': 10, 'max_depth': 10}


In [54]:
rfc2 = RandomForestClassifier(n_estimators=100, min_samples_split=10, max_depth=10, n_jobs=-1, random_state=10)
rfc2.fit(X_train_std, y_train)
pred = rfc2.predict(X_test_std)
print(classification_report(y_test, pred))

              precision    recall  f1-score   support

           0       0.86      0.96      0.91     12463
           1       0.81      0.55      0.66      4201

    accuracy                           0.85     16664
   macro avg       0.84      0.75      0.78     16664
weighted avg       0.85      0.85      0.84     16664



# gridsearch

In [55]:
param_grid2 = dict(max_depth=[8, 9, 10, 11, 12], min_samples_split=[8,10,12], n_estimators=[100, 300, 500])
param_grid2

{'max_depth': [8, 9, 10, 11, 12],
 'min_samples_split': [8, 10, 12],
 'n_estimators': [100, 300, 500]}

In [56]:
gridcv = GridSearchCV(estimator=rfc, param_grid=param_grid2, cv=cv_reg, n_jobs=-1)
gridcv.fit(X_train_std, y_train)
pred = gridcv.predict(X_test_std)
print(classification_report(y_test, pred))
print("best_params: ", gridcv.best_params_)

              precision    recall  f1-score   support

           0       0.87      0.95      0.91     12463
           1       0.81      0.57      0.66      4201

    accuracy                           0.86     16664
   macro avg       0.84      0.76      0.79     16664
weighted avg       0.85      0.86      0.85     16664

best_params:  {'max_depth': 12, 'min_samples_split': 8, 'n_estimators': 300}


In [57]:
rfc3 = RandomForestClassifier(n_estimators=300, min_samples_split=8, max_depth=12, n_jobs=-1, random_state=10)
rfc3.fit(X_train_std, y_train)
pred = rfc3.predict(X_test_std)
print(classification_report(y_test, pred))

              precision    recall  f1-score   support

           0       0.87      0.95      0.91     12463
           1       0.81      0.57      0.66      4201

    accuracy                           0.86     16664
   macro avg       0.84      0.76      0.79     16664
weighted avg       0.85      0.86      0.85     16664



# RandomSearch 평가 기준 변경하기
 이진 분류에서 옵션들

| scoring 값 | 의미 |
|------------|------|
| "f1" | 기본 F1 (양성 클래스 기준) |
| "f1_macro" | 각 클래스 평균 |
| "f1_weighted" | 클래스 비율 가중 평균 |
| "precision" | 정밀도 |
| "recall" | 재현율 |
| "roc_auc" | ROC-AUC |

In [58]:
# f1 score가 높은 파라미터 탐색
randcv = RandomizedSearchCV(estimator=rfc, param_distributions=param_grid,
                            scoring="f1", cv=cv_reg, n_jobs=-1)
randcv.fit(X_train_std, y_train)
pred = randcv.predict(X_test_std)
print(classification_report(y_test, pred))
print("best_params: ", randcv.best_params_)

              precision    recall  f1-score   support

           0       0.86      0.96      0.91     12463
           1       0.81      0.55      0.65      4201

    accuracy                           0.85     16664
   macro avg       0.84      0.75      0.78     16664
weighted avg       0.85      0.85      0.84     16664

best_params:  {'n_estimators': 300, 'min_samples_split': 10, 'max_depth': 10}


In [59]:
# precision score가 높은 파라미터 탐색
randcv = RandomizedSearchCV(estimator=rfc, param_distributions=param_grid,
                            scoring="precision", cv=cv_reg, n_jobs=-1)
randcv.fit(X_train_std, y_train)
pred = randcv.predict(X_test_std)
print(classification_report(y_test, pred))
print("best_params: ", randcv.best_params_)

              precision    recall  f1-score   support

           0       0.83      0.98      0.90     12463
           1       0.87      0.41      0.56      4201

    accuracy                           0.84     16664
   macro avg       0.85      0.70      0.73     16664
weighted avg       0.84      0.84      0.81     16664

best_params:  {'n_estimators': 100, 'min_samples_split': 2, 'max_depth': 5}


In [60]:
# recall score가 높은 파라미터 탐색
randcv = RandomizedSearchCV(estimator=rfc, param_distributions=param_grid,
                            scoring="recall", cv=cv_reg, n_jobs=-1)
randcv.fit(X_train_std, y_train)
pred = randcv.predict(X_test_std)
print(classification_report(y_test, pred))
print("best_params: ", randcv.best_params_)

              precision    recall  f1-score   support

           0       0.86      0.96      0.91     12463
           1       0.81      0.55      0.66      4201

    accuracy                           0.85     16664
   macro avg       0.84      0.75      0.78     16664
weighted avg       0.85      0.85      0.84     16664

best_params:  {'n_estimators': 300, 'min_samples_split': 2, 'max_depth': 10}


In [61]:
# roc_auc score가 높은 파라미터 탐색
randcv = RandomizedSearchCV(estimator=rfc, param_distributions=param_grid,
                            scoring="roc_auc", cv=cv_reg, n_jobs=-1)
randcv.fit(X_train_std, y_train)
pred = randcv.predict(X_test_std)
print(classification_report(y_test, pred))
print("best_params: ", randcv.best_params_)

              precision    recall  f1-score   support

           0       0.86      0.96      0.91     12463
           1       0.81      0.55      0.65      4201

    accuracy                           0.85     16664
   macro avg       0.84      0.75      0.78     16664
weighted avg       0.85      0.85      0.84     16664

best_params:  {'n_estimators': 500, 'min_samples_split': 2, 'max_depth': 10}


# 하이퍼파라미터 튜닝 자동화  FLAML

In [63]:
# !conda install flaml -c conda-forge -y

In [67]:
from flaml import AutoML
from sklearn.metrics import accuracy_score, roc_auc_score

In [71]:
# (필요 시) !pip install flaml


# ===============================
# 🔟-추가) FLAML AutoML 베이스라인
# ===============================
print("\n🚀 FLAML AutoML 학습 중...")

automl = AutoML()
automl.fit(
    X_train_std, y_train,
    task="classification",
    time_budget=60,                 # ⏱️ 총 탐색 시간(초) - 상황에 맞게 조절
    metric="roc_auc",               # 평가지표(불균형이면 'f1'/'roc_auc' 권장)
    eval_method="cv",                # 교차검증
    n_splits=3,
    estimator_list=["lgbm", "xgboost", "rf"],  # 설치된 환경에 맞춰 사용
    verbose=1
)

print("\n✅ FLAML Best Estimator:", automl.best_estimator)
print("✅ FLAML Best Config:", automl.best_config)
print("✅ FLAML Best Loss:", automl.best_loss)
y_pred_flaml = automl.predict(X_test_std)
roc_auc_flaml = roc_auc_score(y_test, y_pred_flaml)
print(f"✅ FLAML 정확도: {roc_auc_flaml:.4f}")
print(classification_report(y_test, y_pred_flaml))



🚀 FLAML AutoML 학습 중...

✅ FLAML Best Estimator: lgbm
✅ FLAML Best Config: {'n_estimators': 281, 'num_leaves': 19, 'min_child_samples': 7, 'learning_rate': np.float64(0.08507396385075812), 'log_max_bin': 7, 'colsample_bytree': np.float64(0.535806469452466), 'reg_alpha': 0.0009765625, 'reg_lambda': np.float64(0.017291604079660877)}
✅ FLAML Best Loss: 0.07533238831378364
✅ FLAML 정확도: 0.8063
              precision    recall  f1-score   support

           0       0.90      0.94      0.92     12463
           1       0.78      0.68      0.73      4201

    accuracy                           0.87     16664
   macro avg       0.84      0.81      0.82     16664
weighted avg       0.87      0.87      0.87     16664

